## Compare results for multiple results for 512 square datasets 
- Aug 11, 2020 : 512 square dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [19]:
%matplotlib widget

In [20]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [21]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [22]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    val_dr=dict_val['hist_val'].copy()
    val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins
    
    sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2
    
    size=len(dict_val['hist_val'])
    l1,l2=int(size*0.3),int(size*0.7)
    keys=['chi_1a','chi_1b','chi_1c','chi_1']
    
    for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
        chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})
    
    idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
#     chisqr_dict.update({'chi_sqr1':})
    
    chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
    chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data
    
    idx=60
    spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})
    
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
    
    return chisqr_dict

In [23]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[5000:8000][:,0,:,:]
print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(3000, 512, 512)


In [24]:
df_runs=pd.DataFrame([])

In [25]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:2000][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [33]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['gen_imgs_1.npy','gen_imgs_2.npy']
for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key='keras_'+strg
    fname=parent_dir+'run2_512_square_20epochs/models/'+fldr
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:2000]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [34]:
# ### Extract a few images generated by Lbann
parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/'

lst=[]
fldr='20200905_080143_bsize10_test/gen_imgs_chkpt/epoch.36.step.81000_20200908_121732/'
lst.append(fldr)
fldr='20200905_080143_bsize20_test/gen_imgs_chkpt/epoch.36.step.40500_20200908_124523/'
lst.append(fldr)
fldr='20200905_080143_bsize25_test/gen_imgs_chkpt/epoch.34.step.30600_20200908_124721/'
lst.append(fldr)
fldr='20200905_080143_bsize50_test/gen_imgs_chkpt/epoch.38.step.17100_20200908_124321/'
lst.append(fldr)
fldr='20200904_200005_bsize100_test/gen_imgs_chkpt/epoch.36.step.8100_20200908_123928/'
lst.append(fldr)

# lst=['20200806_070111_gen_img_exagan/']
names=['1_b10','2_b20','3_b25','4_b50','5_b100']

for strg,fldr in zip(names,lst):
    f_strg=parent_dir+fldr+'dump_outs/trainer0/model0/sgd.testing.epoch.*.step.*_gen_img_instance1_activation_output0.npy'
    f_list=glob.glob(f_strg)
#     print(f_list)
    key='lbann_'+strg
#     fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':f_strg}
    arr=[np.load(fname)[:,0,:,:] for fname in f_list]
    images=np.vstack(arr)
    print(images.shape)
    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(987, 512, 512)
(987, 512, 512)
(987, 512, 512)
(987, 512, 512)
(987, 512, 512)


In [19]:
# ### Extract some LBANN training run data

# parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200805_124242_batchsize_64/dump_outs/trainer0/model0/'
# lst=[(5,16920),(6,20200),(7,23680),(4,14720)]

# for count,run in enumerate(lst):
#     epoch,step=run[0],run[1]
#     key='lbann_train_{0}-{1}'.format(epoch,step)
#     fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
#     dict1={'label':key,'fname':fname}
#     images=np.load(fname)[:,0,:,:]
#     dict1['num_img']=images.shape[0]
#     print(dict1)
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)

# # print(s_lbann.shape,np.max(s_lbann))



In [36]:
print(df_runs.columns)
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'num_img']
df_runs=df_runs.reindex(columns=col_list)

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'fname', 'hist_bin_centers', 'hist_err',
       'hist_val', 'label', 'num_img', 'spec_err', 'spec_val'],
      dtype='object')


In [37]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_img
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0003905041503886839, 0.00028129094188102636...","[1.1193483149075099, 0.9587987861501517, 0.258...","[8106942.17179978, 24994.125958279594, 14433.1...","[51086503832.26049, 2620431.409475068, 2298491...",0.000057,0.000004,0.000007,0.000046,6.596392e-07,1.252274,0.001243,91.740597,2000.0
1,keras_0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0007121786267828354, 0.00036487307095574425...","[1.364978432086031, 0.6150946419267077, 0.2077...","[18932974.16919213, 51903.88286054141, 32605.5...","[54197695384.68548, 3876710.717776271, 3183306...",0.266405,0.189970,0.004115,0.072320,1.809306e-01,2.624038,1.546344,79492.363152,2000.0
2,keras_1,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0011462546590623676, 0.0009407481409491217,...","[1.314546662046793, 0.684971161954853, 0.22301...","[22258166.462042686, 22402.442260187527, 13594...","[53506978267.02771, 1833004.710485073, 1640503...",0.158063,0.117555,0.017563,0.022945,1.142674e-01,2.444335,0.890540,51471.444328,2000.0
3,lbann_1_b10,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0035844461514267134, 0.0017256668753082933,...","[0.929530934945798, 1.1074151556933678, 0.3483...","[75493468.27701892, 129753.18245095655, 102207...","[47326945768.02317, 4560663.244133691, 4299508...",0.364904,0.224253,0.094926,0.045724,7.664110e-02,9.248391,3.703746,56420.205998,987.0
4,lbann_2_b20,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0016262095074084668, 0.0005664741347593807,...","[0.9422502520150479, 1.0555170647775165, 0.360...","[39685996.013978146, 57203.89056811749, 39754....","[47230105892.07414, 3510986.350654594, 3132844...",0.519736,0.317332,0.113059,0.089345,6.775828e-02,5.296557,4.448358,175934.953359,987.0
5,lbann_3_b25,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0016565401790011789, 0.0010536233958434826,...","[1.1073940217461042, 0.9618409748239338, 0.270...","[32739261.494672175, 28242.754166816332, 21969...","[50418827807.96167, 1859384.4413905737, 201222...",0.058176,0.001941,0.000658,0.055577,4.036568e-04,3.936766,0.223370,3166.121382,987.0
6,lbann_4_b50,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.001666989977593401, 0.0014212435839755463, ...","[1.258336090204052, 0.7564017057998595, 0.2443...","[30563153.263993584, 41106.269020450796, 22400...","[52674211854.502075, 2962440.133220066, 233760...",0.113084,0.061333,0.027652,0.024099,6.056632e-02,3.150469,0.152906,10001.233734,987.0
7,lbann_5_b100,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0013518964276029225, 0.0011106943105963483,...","[1.427030731899236, 0.593875472134274, 0.18644...","[27220901.383809745, 26558.852315435994, 17324...","[55873271808.13134, 1490354.4079030557, 127360...",0.489832,0.332991,0.146012,0.010829,2.393637e-01,2.668119,11.565433,414740.702200,987.0


### Compare chi-squares

In [43]:
chi_keys=['label','chi_1','chi_spec1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'num_img','fname']
df_runs[chi_keys]

,label,chi_1,chi_spec1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,num_img,fname
0,raw,0.000057,0.001243,0.000004,0.000007,0.000046,6.596392e-07,1.252274,91.740597,2000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.266405,1.546344,0.189970,0.004115,0.072320,1.809306e-01,2.624038,79492.363152,2000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.158063,0.890540,0.117555,0.017563,0.022945,1.142674e-01,2.444335,51471.444328,2000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,lbann_1_b10,0.364904,3.703746,0.224253,0.094926,0.045724,7.664110e-02,9.248391,56420.205998,987.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...
4,lbann_2_b20,0.519736,4.448358,0.317332,0.113059,0.089345,6.775828e-02,5.296557,175934.953359,987.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...
5,lbann_3_b25,0.058176,0.223370,0.001941,0.000658,0.055577,4.036568e-04,3.936766,3166.121382,987.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...
6,lbann_4_b50,0.113084,0.152906,0.061333,0.027652,0.024099,6.056632e-02,3.150469,10001.233734,987.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...
7,lbann_5_b100,0.489832,11.565433,0.332991,0.146012,0.010829,2.393637e-01,2.668119,414740.702200,987.0,/global/cscratch1/sd/vpa/proj/cosmogan/results...


In [38]:

def f_plot_hist_spec_best(df,dict_bkg):

    fig=plt.figure(figsize=(6,6))
    ax1=fig.add_subplot(121)
    ax2=fig.add_subplot(122)
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

        x1=row.hist_bin_centers
        y1=row.hist_val
        yerr1=row.hist_err
        x1=f_invtransform(x1)

        y2=row.spec_val
        yerr2=row.spec_err
        x2=np.arange(len(y2))

        label=row.label
        ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
    #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

        ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
        ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
    x=f_invtransform(x)
    ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
    ax1.set_title('pixel intensity')
    y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
    x=np.arange(len(y))
    ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
    ax2.set_title('Spectrum')


    plt.legend()
    # plt.yscale('log')
    ax1.set_xscale('symlog',linthreshx=50)
    ax1.set_yscale('log')
    ax2.set_yscale('log')


f_plot_hist_spec_best(df_runs,dict_val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:

def f_plot_hist_spec_best(df,dict_bkg,plot_type):

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title("Spectrum")
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
#     ax1.set_xscale('symlog',linthreshx=50)
#     ax1.set_yscale('log')
#     ax2.set_yscale('log')


f_plot_hist_spec_best(df_runs,dict_val,'hist')
f_plot_hist_spec_best(df_runs,dict_val,'spec')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
def f_widget_compare(sample_names,sample_dict,Fig_type='pixel',rescale=True,log_scale=True,bins=25,mode='avg',normalize=True,bkgnd=[]):
    '''
    Module to make widget plots for pixel intensity or spectrum comparison for multiple sample sets
    '''
    
    ### Crop out large pixel values
#     for key in sample_names:
#         print(sample_dict[key].shape)
#         sample_dict[key]=np.array([arr for arr in sample_dict[key] if np.max(arr)<=0.994])
#         print(sample_dict[key].shape)
    
    img_list=[sample_dict[key] for key in sample_names]
    label_list=list(sample_names)
    
    if rescale: 
        for count,img in enumerate(img_list):
            img_list[count]=f_invtransform(img)
        if len(bkgnd): bkgnd=f_invtransform(bkgnd)
#         hist_range=(0,2000)
    else:
        bins=f_transform(bins)
#         hist_range=(-1,0.996)
    assert Fig_type in ['pixel','spectrum'],"Invalid mode %s"%(mode)
    
    if Fig_type=='pixel':
#         f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=hist_range)
        f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=None,bkgnd_arr=bkgnd)

    elif Fig_type=='spectrum':
        f_compare_spectrum(img_lst=img_list,label_lst=label_list,log_scale=log_scale,bkgnd_arr=bkgnd)



In [42]:
# dict_samples={'raw': s_raw,'keras1':s_keras[0],'keras2':s_keras[1],'s_new1':s_new1,'s_new2':s_new2,'s_new3':s_new3}

# bkgnd=s_raw
# bkgnd=[]
interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),
                bins=fixed(bins),
                mode=['avg','simple'],bkgnd=fixed(bkgnd))

NameError: name 'dict_samples' is not defined